In [ ]:
"""
01, cap at .99, 0.387178, 0.384541
"""

In [ ]:
"""
categorical only + various estimator
numerical only + various estimator
variable selection (xgb, RF) + various estimator
numerical as categorical
second order features
standardize kfold transformer
numerical as categorical + 2nd order + linear
"""

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xgboost as xgb
import vw_utils as vw
import gini
%matplotlib inline

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

cap = train.Hazard.quantile(0.99)
print(cap)

train['Hazard_cap'] = np.minimum(train.Hazard, cap)

data = pd.concat([train, test], axis=0).reset_index()
train_set = np.hstack([np.ones(train.shape[0]), np.zeros(test.shape[0])])

19.0


## --------------------- Feature Start -----------------------

In [3]:
# start
categorical = ['T1_V' + str(i) for i in list(range(4, 10)) + [11, 12, 15, 16, 17]] + \
                ['T2_V' + str(i) for i in [3, 5, 11, 12, 13]]

numerical = set(train.columns).difference(categorical + ['Hazard', 'Hazard_cap', 'Id'])
numerical = list(numerical)

In [4]:
# raw feature
from sklearn.preprocessing import StandardScaler
import re

SS = StandardScaler()
feat_raw = data[numerical].applymap(float)
feat_raw[numerical] = SS.fit_transform(feat_raw[numerical])


toDrop = re.sub('[ ]+', '_', data[categorical].ix[0, :].to_string()).split('\n')
feat_raw_cat = pd.get_dummies(data[categorical]).drop(toDrop, axis=1)

feat_raw = pd.concat([feat_raw, feat_raw_cat], axis=1)
feat_raw.shape

(101999, 95)

In [20]:
%%time
# count features
from wrappers import generatePrimes, calPowerCount
from sklearn.preprocessing import StandardScaler

feat_count = pd.concat([calPowerCount(data[categorical], 1), 
                        calPowerCount(data[categorical], 2),
                        calPowerCount(data[categorical], 3)], axis=1)

cols = feat_count.columns
feat_count = pd.DataFrame(StandardScaler().fit_transform(feat_count.values), columns=cols)

CPU times: user 14.7 s, sys: 1.23 s, total: 16 s
Wall time: 16 s


/home/albertium/anaconda3/lib/python3.4/site-packages/sklearn/utils/validation.py:498: UserWarning: StandardScaler assumes floating point values as input, got int64
  "got %s" % (estimator, X.dtype))


In [23]:
# assemble features

tmp = pd.concat([feat_count], axis=1)
train_c = tmp[train_set==1]
test_c = tmp[train_set==0]

print(train_c.shape)
print(test_c.shape)

(50999, 968)
(51000, 968)


## --------------------- Modeling -----------------------

In [24]:
%%time

import gini
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import KFold

param = {'max_depth':7, 'eta':0.01, 'objective':'reg:linear',
         'gamma':0, 'min_child_weight': 30, 'subsample': 0.8,
         'colsample_bytree': 0.5}

scores = []
time = 1
for idx_train, idx_test in KFold(train.shape[0], 2):
    print(time)
    time += 1
    dTrain = xgb.DMatrix(train_c.ix[idx_train], label=train.Hazard_cap[idx_train])
    dTest = xgb.DMatrix(train_c.ix[idx_test])
    model = xgb.train(param, dTrain, 1000)
    pred = model.predict(dTest)
    scores.append(gini.normalized_gini(train.Hazard[idx_test], pred))

print(np.mean(scores))

1
2
3
4
5
6
7
8
9
10
0.293445345882
CPU times: user 4h 54min 13s, sys: 15.2 s, total: 4h 54min 28s
Wall time: 1h 18min 14s


In [13]:
%%time

import gini
import xgboost as xgb

param = {'max_depth':7, 'eta':0.01, 'objective':'reg:linear',
         'gamma':0, 'min_child_weight': 30, 'subsample': 0.8,
         'colsample_bytree': 0.5}

dTrain = xgb.DMatrix(train_c, label=train.Hazard_cap)
dTest = xgb.DMatrix(test_c)
model = xgb.train(param, dTrain, 1000)
yhat = model.predict(dTest)

(pd.DataFrame({'Id': test.Id, 'Hazard': yhat})
 .reindex_axis(['Id', 'Hazard'], 1)
 .to_csv('../output/xgb_v2_01.csv', index=0))

CPU times: user 59.4 s, sys: 180 ms, total: 59.6 s
Wall time: 16.1 s


In [44]:
pd.DataFrame({'Id': test.Id, 'Hazard': yhat}).reindex_axis(['Id', 'Hazard'], 1).to_csv('../output/ensemble_07.csv', index=0)